In [1]:
import torch
from sparse_tensor import SparseTensor
from sparsify.sparsify import Sae

In [2]:
sae = Sae.load_from_hub("EleutherAI/sae-pythia-410m-65k", hookpoint="layers.22.mlp")
sae = sae.to('cuda:0')

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [3]:
ip = torch.rand((512,1024), device='cuda:0')

In [4]:
sae.cfg

SparseCoderConfig(activation='topk', expansion_factor=32, normalize_decoder=True, num_latents=65536, k=32, multi_topk=False, skip_connection=False, transcode=False)

In [5]:
with torch.no_grad():
    result = sae.forward(ip)

In [6]:
st = SparseTensor(result.latent_acts, result.latent_indices, (512,sae.num_latents), 1)

In [8]:
full = SparseTensor.decompress(st)

Dense Tensor Created: torch.Size([512, 65536])
1
tensor([[ 4353,  8129, 10602,  ..., 57425, 58835, 39907],
        [ 4353,  8129, 10602,  ..., 57425, 58835,  2730],
        [ 2907,  4353, 10690,  ..., 57425, 58835, 28422],
        ...,
        [ 4353,  4997, 10602,  ..., 57425, 58835, 55763],
        [ 4353,  8129, 10602,  ..., 57425, 58835, 39907],
        [ 2706,  4353, 10602,  ..., 57425, 58835, 53706]], device='cuda:0')
tensor([[8.5625, 4.5312, 4.4375,  ..., 4.4062, 4.7188, 3.9844],
        [9.1250, 4.6562, 4.7812,  ..., 5.3750, 4.5312, 3.9844],
        [4.2500, 8.3750, 9.9375,  ..., 4.7500, 4.7812, 4.0000],
        ...,
        [8.3750, 3.9688, 4.7500,  ..., 5.1875, 4.1875, 3.9375],
        [8.3125, 4.3125, 4.7500,  ..., 4.9062, 4.6562, 4.1250],
        [4.1250, 8.1875, 4.5938,  ..., 4.6562, 4.2188, 4.0938]],
       device='cuda:0', dtype=torch.bfloat16)


In [9]:
full.shape

torch.Size([512, 65536])

In [10]:
check = (full != 0)
print(check.sum(dim=-1).max())
print(check.sum(dim=-1).min())

tensor(32, device='cuda:0')
tensor(32, device='cuda:0')
